In [60]:
# Have to install janitor package to run. Not included in anaconda.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor

In [61]:
exec(open("../header.py").read())

# Purpose

To replicate and then test out Boone and White (2015).

# Import raw data

In [62]:
sample = True
header = Header(sample)

In [63]:
raw_russ = pd.read_csv(header.raw_root("russell_index_inclusion.csv"), parse_dates = ['Date'])

In [64]:
raw_russ.head(5)

,Date,CUSIP,Ticker,MktValue,Shares,Russell1000,Russell2000,Name,R1000_WT,R2000_WT
0,1996-07-31,302426101,FHPC,NaN,40682,Y,N,FHP INTERNATIONAL CO,0.020100,NaN
1,2001-05-31,457153104,IM,879349.50,66366,Y,N,INGRAM MICRO INC,0.000078,NaN
2,2004-11-30,949475107,WC,1107957.84,22616,Y,N,WELLCHOICE INC,0.000095,NaN
3,1999-11-30,553087107,MMI,73669.37,18134,N,Y,MMI COS INC,NaN,0.000086
4,1996-10-31,500228101,KE,NaN,16104,N,Y,KOGER EQUITY INC,NaN,0.038500


# Clean data

In [65]:
cleaned_russ = raw_russ\
    .clean_names()\
    .loc[lambda x:(x.date >= '1996-01-01')&(x.date <= '2006-12-31')]\
    .assign(russell1000 = lambda x:np.select(
        [x.russell1000 == 'Y',x.russell1000 == 'N'],
        [1,0],
        default = -1),
            russell2000 = lambda x:np.select(
        [x.russell2000 == 'Y',x.russell2000 == 'N'],
        [1,0],
        default = -1),
            r1000_wt = lambda x:x.r1000_wt.fillna(0),
            r2000_wt = lambda x:x.r2000_wt.fillna(0)
    )\
    .assign(year = lambda x:x.date.dt.year,
            month = lambda x:x.date.dt.month,
            yrmo = lambda x: x.year*100 + x.month)

In [66]:
cleaned_russ.head(5)

,date,cusip,ticker,mktvalue,shares,russell1000,russell2000,name,r1000_wt,r2000_wt,year,month,yrmo
0,1996-07-31,302426101,FHPC,NaN,40682,1,0,FHP INTERNATIONAL CO,0.020100,0.000000,1996,7,199607
1,2001-05-31,457153104,IM,879349.50,66366,1,0,INGRAM MICRO INC,0.000078,0.000000,2001,5,200105
2,2004-11-30,949475107,WC,1107957.84,22616,1,0,WELLCHOICE INC,0.000095,0.000000,2004,11,200411
3,1999-11-30,553087107,MMI,73669.37,18134,0,1,MMI COS INC,0.000000,0.000086,1999,11,199911
4,1996-10-31,500228101,KE,NaN,16104,0,1,KOGER EQUITY INC,0.000000,0.038500,1996,10,199610


# Data integrity checks

In [67]:
assert len(cleaned_russ.russell1000.unique()) == 2
assert len(cleaned_russ.russell2000.unique()) == 2

print("All good")

All good


In [68]:
print(f"Old shape: {raw_russ.shape}")
print(f"New shape: {cleaned_russ.shape}")

Old shape: (10000, 10)
New shape: (10000, 13)


# Save cleaned datsets

In [69]:
cleaned_russ.to_csv(header.clean_root("russell.csv"), index = False)